# Deep Learning Toolkit for Splunk - Spark MLLib Collaborative Filtering

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import sys
import json
import pandas as pd
import numpy as np
from random import random
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
SparkSession

numpy version: 1.18.4
pandas version: 1.0.3


pyspark.sql.session.SparkSession

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| inputlookup supermarket.csv<br> 
| stats count by customer_id product_id <br>
| fit MLTKContainer mode=stage algo=spark_recommender rank=10 numIterations=10 product_id from customer_id into app:recommendation <br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [13]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [19]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("recommendation")
print(param)
df

{'options': {'params': {'mode': 'stage', 'algo': 'spark_recommender', 'rank': '10', 'numIterations': '10'}, 'args': ['customer_id', 'product_id', 'count'], 'feature_variables': ['customer_id', 'product_id', 'count'], 'model_name': 'recommendation', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '1000', 'max_distinct_cat_values_for_classifiers': '1000', 'max_distinct_cat_values_for_scoring': '1000', 'max_fit_time': '6000', 'max_inputs': '100000000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '150', 'max_score_time': '6000', 'streaming_apply': '0', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['customer_id', 'product_id', 'count']}


,customer_id,product_id,count
0,1,1004,1
1,1,1007,1
2,1,1010,2
3,1,1011,1
4,1,1111,1
...,...,...,...
82147,98,3664,1
82148,98,370,1
82149,98,463,1
82150,98,476,1


In [5]:
print(param['options']['model_name'])

recommendation


In [6]:
print(param['feature_variables'])
print(param['target_variables'][0])

['customer_id']
product_id


## Stage 2 - create and initialize a model

In [7]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    appName = "recommendation"
    if 'options' in param:
        if 'model_name' in param['options']: 
            appName = param['options']['model_name']
    sparkConf = SparkConf().setAll([('spark.executor.memory', '1g'), ('spark.executor.cores', '1'), ('spark.cores.max', '4'), ('spark.driver.memory','4g'), ('spark.driver.maxResultSize','4g')])
    spark = SparkSession.builder.appName(appName).config(conf=sparkConf).getOrCreate()
    model['spark'] = spark
    return model

In [8]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'spark': <pyspark.sql.session.SparkSession object at 0x7fd7ae669e10>}


In [9]:
model['spark']

In [10]:
str(model['spark'].sparkContext.getConf().getAll())

"[('spark.driver.host', 'b076ba5841a7'), ('spark.driver.memory', '4g'), ('spark.executor.id', 'driver'), ('spark.cores.max', '4'), ('spark.executor.memory', '1g'), ('spark.rdd.compress', 'True'), ('spark.driver.port', '39187'), ('spark.serializer.objectStreamReset', '100'), ('spark.app.id', 'local-1592455652680'), ('spark.master', 'local[*]'), ('spark.executor.cores', '1'), ('spark.submit.deployMode', 'client'), ('spark.app.name', 'recommendation'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.maxResultSize', '4g')]"

## Stage 3 - fit the model

In [118]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    spark = model['spark']
    sc = spark.sparkContext
    feature_variables = param['feature_variables']
    
    rank=10
    numIterations=10
    if 'options' in param:
        if 'params' in param['options']:
            if 'rank' in param['options']['params']:
                rank = int(param['options']['params']['rank'])
            if 'numIterations' in param['options']['params']:
                numIterations = int(param['options']['params']['numIterations'])

    sdf = spark.createDataFrame(df)
    ratings = sdf.rdd.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
    model['als'] = ALS.train(ratings, rank, numIterations)

    testdata = ratings.map(lambda p: (p[0], p[1]))
    predictions = model['als'].predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
    ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
    MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
    model['MSE'] = MSE
    info = {"message": "model trained", "Mean Squared Error": str(MSE)}
    return info

In [119]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained', 'Mean Squared Error': '0.08126842820906775'}


## Stage 4 - apply the model

In [107]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    spark = model['spark']
    sdf = spark.createDataFrame(df)
    limit = None
    if 'options' in param:
        if 'params' in param['options']:
            if 'limit' in param['options']['params']:
                limit = int(param['options']['params']['limit'])
    ratings = sdf.rdd.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
    testdata = ratings.map(lambda p: (p[0], p[1]))
    predictions = model['als'].predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
    if limit==None:
        results = predictions.map(lambda r: (r[1])).collect()
    else:
        results = predictions.takeOrdered(limit, key = lambda x: -x[1])
    result = pd.DataFrame(results)
    return result

In [108]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

              0
0      1.198177
1      1.054113
2      0.959596
3      0.818107
4      1.040066
...         ...
82147  0.980672
82148  1.011859
82149  0.979656
82150  0.987319
82151  0.989235

[82152 rows x 1 columns]


## Stage 5 - save the model

In [125]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    sc = model['spark'].sparkContext
    import shutil
    from pathlib import Path
    if Path(MODEL_DIRECTORY + name).is_dir():
        shutil.rmtree(MODEL_DIRECTORY + name)
    model['als'].save(sc, MODEL_DIRECTORY + name)
    return model

In [129]:
save(model,'recommendation_system')

{'spark': <pyspark.sql.session.SparkSession at 0x7fd7ae669e10>,
 'als': <pyspark.mllib.recommendation.MatrixFactorizationModel at 0x7fd7ac2fb790>}

## Stage 6 - load the model

In [123]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = init({},{})
    model['als'] = MatrixFactorizationModel.load(model['spark'].sparkContext, MODEL_DIRECTORY + name) 
    return model

In [124]:
model = load('recommendation_system')
model

{'spark': <pyspark.sql.session.SparkSession at 0x7fd7ae669e10>,
 'als': <pyspark.mllib.recommendation.MatrixFactorizationModel at 0x7fd7ac2fb790>}

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"spark": "no model"}
    if model:
        returns = {"spark_info": str(model['spark'].sparkContext.getConf().getAll()) }
    return returns

In [ ]:
summary(model)

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code